In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Visit the Mars Facts site
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
browser.visit(url)

html = browser.html
html_soup = soup(html, 'html.parser')

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 12.1MB/s]


In [2]:
table = html_soup.find('table', class_='tableList')

In [94]:
import re

best_books = {}
rows = table.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    num = int(cols[0].text)
    info = cols[2]
    name = info.a.text.strip()
    auth = info.findNext(class_='authorName').text
    stars = info.findNext(class_='minirating').text
    rating = re.findall(r'\d+\.\d+', stars)[0]
    best_books[num] = [name, auth, float(rating)]

In [93]:
best_books

{1: ['The Hunger Games (The Hunger Games, #1)', 'Suzanne Collins', 4.33],
 2: ['Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
  'J.K. Rowling',
  4.5],
 3: ['Pride and Prejudice', 'Jane Austen', 4.28],
 4: ['To Kill a Mockingbird', 'Harper Lee', 4.27],
 5: ['The Book Thief', 'Markus Zusak', 4.39],
 6: ['Twilight (The Twilight Saga, #1)', 'Stephenie Meyer', 3.64],
 7: ['Animal Farm', 'George Orwell', 3.98],
 8: ['The Chronicles of Narnia (Chronicles of Narnia, #1-7)',
  'C. S. Lewis',
  4.27],
 9: ['J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings',
  'J.R.R. Tolkien',
  4.61],
 10: ['The Fault in Our Stars', 'John Green', 4.15],
 11: ['Gone with the Wind', 'Margaret Mitchell', 4.3],
 12: ['The Giving Tree', 'Shel Silverstein', 4.38],
 13: ['Wuthering Heights', 'Emily Brontë', 3.88],
 14: ['The Picture of Dorian Gray', 'Oscar Wilde', 4.12],
 15: ['Jane Eyre', 'Charlotte Brontë', 4.14],
 16: ['The Perks of Being a Wallflower', 'Stephen Chbosky', 4.22

In [84]:
starz

['4.33',
 '4.50',
 '4.28',
 '4.27',
 '4.39',
 '3.64',
 '3.98',
 '4.27',
 '4.61',
 '4.15',
 '4.30',
 '4.38',
 '3.88',
 '4.12',
 '4.14',
 '4.22',
 '3.90',
 '3.93',
 '4.14',
 '4.07',
 '4.32',
 '4.15',
 '4.19',
 '4.25',
 '4.29',
 '3.69',
 '3.74',
 '4.30',
 '4.08',
 '4.23',
 '4.47',
 '4.31',
 '4.19',
 '4.01',
 '3.88',
 '3.90',
 '3.99',
 '3.81',
 '3.99',
 '4.11',
 '3.97',
 '4.42',
 '4.26',
 '4.15',
 '4.44',
 '4.12',
 '3.99',
 '3.80',
 '3.83',
 '3.84',
 '4.23',
 '4.33',
 '4.30',
 '4.13',
 '4.19',
 '3.93',
 '3.85',
 '3.86',
 '4.13',
 '4.13',
 '4.62',
 '3.88',
 '4.47',
 '4.10',
 '4.09',
 '4.47',
 '4.26',
 '4.34',
 '3.99',
 '4.04',
 '4.33',
 '4.58',
 '4.33',
 '4.30',
 '4.08',
 '3.79',
 '4.24',
 '4.25',
 '4.16',
 '4.38',
 '4.20',
 '4.25',
 '4.09',
 '4.08',
 '4.35',
 '4.29',
 'real',
 '3.98',
 '4.14',
 '4.11',
 '4.06',
 '4.01',
 '4.08',
 '4.25',
 '4.28',
 '4.26',
 '4.38',
 '4.01',
 '4.27',
 '3.89']